In [1]:
import requests
import pyperclip
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from pathlib import Path
import os
import re
import time

In [2]:

def getLinks(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    searchResults = soup.find_all('div',{'class':'lessonTitle'})
    html = ''
    for tag in searchResults:
        #print(str(tag))
        pid = tag.parent.parent
        link = str(pid.find('a').get('href'))
        level = pid.find('div').text
        level = level[6:]
        html += "https://gloss.dliflc.edu/"+str(link)+":::"+level+"\n"
    return html

def waitForAlert(driver):
    try:
        WebDriverWait(driver, 1).until(EC.alert_is_present(),
                                       'Timed out waiting for PA creation ' +
                                       'confirmation popup to appear.')

        alert = driver.switch_to.alert
        alert.accept()
        print("alert accepted")
        waitForAlert(driver)
    except TimeoutException:
        print("no alert")

def getTextFromLink(url, difficulty, level, count):
    dir = 'C:\\Users\\maste\\Desktop\\arabicScraperOutput'
    filename = "aj_" + str(level)+"_"+str(count) + ".txt"
    #print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content,"html.parser")

    ps = soup.find("div",{"class":"formilized-body"})
    
    if ps != None:
        myFileName = os.path.join(dir, filename)
        fout = open(myFileName, "w+",encoding='utf8')
        fout.write("<DIFFICULTY>"+str(difficulty)+"</DIFFICULTY>\n")
        fout.write("<BODY>\n")
        fout.write(ps.text.replace("\n\n","\n") + "\n")
        fout.write("</BODY>\n")
        fout.close()
    else:
        print("no text found")
        

In [11]:
def getModules(level, difficulty):
    html = ''
    links = {}
    driver = webdriver.Chrome()
    driver.get("http://learning.aljazeera.net/en/languageofmedia/level/"+level)
    driver.implicitly_wait(1)
    next = ''
    while (str(next) != 'None'):
        checkHtml = driver.page_source
        soup = BeautifulSoup(checkHtml, 'html.parser')
        parentDiv = soup.find('div',{'class':'has-pagination-contaienr'})
        headers = parentDiv.find_all('h2')
        for h in headers:
            link = h.find('a')
            url = "http://learning.aljazeera.net" + link['href']
            print(url)
            if url in links:
                print('duplicate')
            else:
                links[url] = difficulty
            html += url + "\n"
        #figure out if on the last page
        ul = soup.find('ul',{'class':'pagination'})
        lis = ul.find_all('li')
        lastLi = lis[len(lis) - 2]
        if lastLi['class'][0] == 'active':
            next = 'None'
        else:
            button = driver.find_element_by_class_name('next-page')
            actions = ActionChains(driver)
            actions.move_to_element(button).perform()
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            button.click()
            time.sleep(1)
    count = 0
    for key in links:
        getTextFromLink(key, difficulty, level, count)
        count += 1
    print(count)
    return html

In [12]:
if __name__ == "__main__":
    html = {}
    print("BEGINNER")
    html['beginner'] = getModules('beginner', '1')
    print("ELEMENTARY")
    html['elementary'] = getModules('elementary', '2')
    print("INTERMEDIATE")
    html['intermediate'] = getModules('intermediate', '3')
    print("UPPER")
    html['upperintermediate'] = getModules('upperintermediate', '4')
    print("ADVANCEDR")
    html['advanced'] = getModules('advanced','5')

BEGINNER
http://learning.aljazeera.net/en//languageofmedia/مقهى-الجزيرة-يستضيف-المدرب-العراقي-عدنان-حمد
http://learning.aljazeera.net/en//languageofmedia/مقهى-الجزيرة-يستضيف-المعلق-خليل-البلوشي
http://learning.aljazeera.net/en//languageofmedia/أهلا-بكم-في-مقهى-الجزيرة-الحلقة-3
http://learning.aljazeera.net/en//languageofmedia/أهلا-بكم-في-مقهى-الجزيرة-الحلقة-2
http://learning.aljazeera.net/en//languageofmedia/معرض-صور-لمعاناة-اللاجئين-السوريين
http://learning.aljazeera.net/en//languageofmedia/نظارات-لتفادي-النوم-أثناء-القيادة
http://learning.aljazeera.net/en//languageofmedia/أهلا-وسهلا-بكم-في-مقهى-الجزيرة
http://learning.aljazeera.net/en//languageofmedia/معرض-فرانكفورت-للكتاب
http://learning.aljazeera.net/en//languageofmedia/فيروس-لمحاربة-سرطان-الدماغ
http://learning.aljazeera.net/en//languageofmedia/مقهى-الجزيرة-يستضيف-المدرب-العراقي-عدنان-حمد
duplicate
http://learning.aljazeera.net/en//languageofmedia/مقهى-الجزيرة-يستضيف-المعلق-خليل-البلوشي
duplicate
http://learning.aljazeera.net/en//

http://learning.aljazeera.net/en//languageofmedia/مخترع-فلبيني-يبتكر-سيارة-طائرة
duplicate
http://learning.aljazeera.net/en//languageofmedia/التبلدي-شجرة-مثمرة-تعيش-مئات-السنين
duplicate
http://learning.aljazeera.net/en//languageofmedia/الأزهار-البرية-تتفتح-بغرب-أستراليا
duplicate
http://learning.aljazeera.net/en//languageofmedia/جوز-الهند-بمدينة-صلالة-العمانية
duplicate
http://learning.aljazeera.net/en//languageofmedia/معرض-الكتاب-العربي-في-إسطنبول
duplicate
http://learning.aljazeera.net/en//languageofmedia/السماء-تمطر-سمكا
duplicate
http://learning.aljazeera.net/en//languageofmedia/الممثلة-جمانة-سعيدة-بأداء-الحج
duplicate
http://learning.aljazeera.net/en//languageofmedia/الطائرات-الورقية-تلون-سماء-سريلانكا
duplicate
http://learning.aljazeera.net/en//languageofmedia/المسلمون-يحتفلون-بعيد-الأضحى-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/سوق-النحاس-بغازي-عنتاب
duplicate
http://learning.aljazeera.net/en//languageofmedia/العراق-يسمح-باستيراد-الخضراوات
duplicate
http://

http://learning.aljazeera.net/en//languageofmedia/مخترع-فلبيني-يبتكر-سيارة-طائرة
duplicate
http://learning.aljazeera.net/en//languageofmedia/التبلدي-شجرة-مثمرة-تعيش-مئات-السنين
duplicate
http://learning.aljazeera.net/en//languageofmedia/الأزهار-البرية-تتفتح-بغرب-أستراليا
duplicate
http://learning.aljazeera.net/en//languageofmedia/جوز-الهند-بمدينة-صلالة-العمانية
duplicate
http://learning.aljazeera.net/en//languageofmedia/معرض-الكتاب-العربي-في-إسطنبول
duplicate
http://learning.aljazeera.net/en//languageofmedia/السماء-تمطر-سمكا
duplicate
http://learning.aljazeera.net/en//languageofmedia/الممثلة-جمانة-سعيدة-بأداء-الحج
duplicate
http://learning.aljazeera.net/en//languageofmedia/الطائرات-الورقية-تلون-سماء-سريلانكا
duplicate
http://learning.aljazeera.net/en//languageofmedia/المسلمون-يحتفلون-بعيد-الأضحى-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/سوق-النحاس-بغازي-عنتاب
duplicate
http://learning.aljazeera.net/en//languageofmedia/العراق-يسمح-باستيراد-الخضراوات
duplicate
http://

http://learning.aljazeera.net/en//languageofmedia/مخترع-فلبيني-يبتكر-سيارة-طائرة
duplicate
http://learning.aljazeera.net/en//languageofmedia/التبلدي-شجرة-مثمرة-تعيش-مئات-السنين
duplicate
http://learning.aljazeera.net/en//languageofmedia/الأزهار-البرية-تتفتح-بغرب-أستراليا
duplicate
http://learning.aljazeera.net/en//languageofmedia/جوز-الهند-بمدينة-صلالة-العمانية
duplicate
http://learning.aljazeera.net/en//languageofmedia/معرض-الكتاب-العربي-في-إسطنبول
duplicate
http://learning.aljazeera.net/en//languageofmedia/السماء-تمطر-سمكا
duplicate
http://learning.aljazeera.net/en//languageofmedia/الممثلة-جمانة-سعيدة-بأداء-الحج
duplicate
http://learning.aljazeera.net/en//languageofmedia/الطائرات-الورقية-تلون-سماء-سريلانكا
duplicate
http://learning.aljazeera.net/en//languageofmedia/المسلمون-يحتفلون-بعيد-الأضحى-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/سوق-النحاس-بغازي-عنتاب
duplicate
http://learning.aljazeera.net/en//languageofmedia/العراق-يسمح-باستيراد-الخضراوات
duplicate
http://

http://learning.aljazeera.net/en//languageofmedia/مخترع-فلبيني-يبتكر-سيارة-طائرة
duplicate
http://learning.aljazeera.net/en//languageofmedia/التبلدي-شجرة-مثمرة-تعيش-مئات-السنين
duplicate
http://learning.aljazeera.net/en//languageofmedia/الأزهار-البرية-تتفتح-بغرب-أستراليا
duplicate
http://learning.aljazeera.net/en//languageofmedia/جوز-الهند-بمدينة-صلالة-العمانية
duplicate
http://learning.aljazeera.net/en//languageofmedia/معرض-الكتاب-العربي-في-إسطنبول
duplicate
http://learning.aljazeera.net/en//languageofmedia/السماء-تمطر-سمكا
duplicate
http://learning.aljazeera.net/en//languageofmedia/الممثلة-جمانة-سعيدة-بأداء-الحج
duplicate
http://learning.aljazeera.net/en//languageofmedia/الطائرات-الورقية-تلون-سماء-سريلانكا
duplicate
http://learning.aljazeera.net/en//languageofmedia/المسلمون-يحتفلون-بعيد-الأضحى-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/سوق-النحاس-بغازي-عنتاب
duplicate
http://learning.aljazeera.net/en//languageofmedia/العراق-يسمح-باستيراد-الخضراوات
duplicate
http://

...

http://learning.aljazeera.net/en//languageofmedia/نصائح-لتعلم-اللغات-الأجنبية
duplicate
http://learning.aljazeera.net/en//languageofmedia/الجزيرة-تتسلم-جائزة-الاتصالات-الدولية
duplicate
http://learning.aljazeera.net/en//languageofmedia/لغة-الجسد-تكشف-ما-يخفيه-الآخرون
duplicate
http://learning.aljazeera.net/en//languageofmedia/بنود-الاتفاق-الروسي-التركي-بشأن-إدلب-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/احتفال-الشعلة-الشهير-في-كوريا-الشمالية-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/أخطاء-مقابلات-العمل-الشخصية
duplicate
http://learning.aljazeera.net/en//languageofmedia/رحلة-إلى-نهر-القاش-في-السودان-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/بخارى-عاصمة-صناعة-السجاد-التركماني
duplicate
http://learning.aljazeera.net/en//languageofmedia/التهنئة-بالعيد-عبر-التواصل-الاجتماعي
duplicate
http://learning.aljazeera.net/en//languageofmedia/أكبر-مندي-أشهر-سوق-للبهارات-بالعالم-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/ان

http://learning.aljazeera.net/en//languageofmedia/نصائح-لتعلم-اللغات-الأجنبية
duplicate
http://learning.aljazeera.net/en//languageofmedia/الجزيرة-تتسلم-جائزة-الاتصالات-الدولية
duplicate
http://learning.aljazeera.net/en//languageofmedia/لغة-الجسد-تكشف-ما-يخفيه-الآخرون
duplicate
http://learning.aljazeera.net/en//languageofmedia/بنود-الاتفاق-الروسي-التركي-بشأن-إدلب-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/احتفال-الشعلة-الشهير-في-كوريا-الشمالية-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/أخطاء-مقابلات-العمل-الشخصية
duplicate
http://learning.aljazeera.net/en//languageofmedia/رحلة-إلى-نهر-القاش-في-السودان-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/بخارى-عاصمة-صناعة-السجاد-التركماني
duplicate
http://learning.aljazeera.net/en//languageofmedia/التهنئة-بالعيد-عبر-التواصل-الاجتماعي
duplicate
http://learning.aljazeera.net/en//languageofmedia/أكبر-مندي-أشهر-سوق-للبهارات-بالعالم-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/ان

http://learning.aljazeera.net/en//languageofmedia/نصائح-لتعلم-اللغات-الأجنبية
duplicate
http://learning.aljazeera.net/en//languageofmedia/الجزيرة-تتسلم-جائزة-الاتصالات-الدولية
duplicate
http://learning.aljazeera.net/en//languageofmedia/لغة-الجسد-تكشف-ما-يخفيه-الآخرون
duplicate
http://learning.aljazeera.net/en//languageofmedia/بنود-الاتفاق-الروسي-التركي-بشأن-إدلب-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/احتفال-الشعلة-الشهير-في-كوريا-الشمالية-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/أخطاء-مقابلات-العمل-الشخصية
duplicate
http://learning.aljazeera.net/en//languageofmedia/رحلة-إلى-نهر-القاش-في-السودان-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/بخارى-عاصمة-صناعة-السجاد-التركماني
duplicate
http://learning.aljazeera.net/en//languageofmedia/التهنئة-بالعيد-عبر-التواصل-الاجتماعي
duplicate
http://learning.aljazeera.net/en//languageofmedia/أكبر-مندي-أشهر-سوق-للبهارات-بالعالم-1
duplicate
http://learning.aljazeera.net/en//languageofmedia/ان

no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
no text found
135
UPPER
http://learning.aljazeera.net/en//languageofmedia/هل-أصبح-فيسبوك-منصة-شائخة
http://learning.aljazeera.net/en//languageofmedia/رحلة-العود-1
http://learning.aljazeera.net/en//languageofmedia/عندما-تصبح-الضباع-حيوانات-أليفة
http://learning.aljazeera.net/en//languageofmedia/انتخابات-باكستان-سلطة-متغيرة-وسياسة-ثابتة
http://learning.aljazeera.net/en//languageofmedia/الانتخابات-التركية-الأهمية-والتوقعات-والانعكاسات
http://learning.aljazeera.net/en//languageofmedia/معاهد-العربية-بالسنغال-روافد-للشريعة-وجسر-مع-العرب
http://learning.aljazeera.net/en//languageofmedia/ما-فوائد-الصيام
http://learning.aljazeera.net/en//languageofmedia/مسيرات-العودة-ما-قبل-الانفجار-الوشيك-1
http://learning.aljazeera.net/en//languageofmedia/بوادر-

IndexError: list index out of range